In [1]:
import pandas as pd
import numpy as np
import copernicusmarine as cm

/home/kshitiz/anaconda3/envs/new_atlantis/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cm.login(username="", password="")

File /home/kshitiz/.copernicusmarine/.copernicusmarine-credentials already exists, overwrite it ? [y/N]:

INFO - 2024-08-06T17:19:24Z - Credentials file stored in /home/kshitiz/.copernicusmarine/.copernicusmarine-credentials.


True

Dataset 5: Bathymetry (Sea floor depth below geoid) data from Global Ocean Physics Reanalysis (https://data.marine.copernicus.eu/product/GLOBAL_MULTIYEAR_PHY_001_030/description)
- dataset_id: "cmems_mod_glo_phy_my_0.083deg_static"
- variables: ["deptho"]
- Parameter definition:
    - deptho [m]: Sea floor depth below geoid
- Spatial resolution: 0.083deg x 0.083deg
Also need to specify min and max depth parameters for getting bathymetry data
- depth: [0.49402499198913574, 5727.9169921875]

In [3]:
# filename for saving the data as csv
filename = 'bathy'

In [4]:
# Set parameters
data_request = {
    "dataset_id" : "cmems_mod_glo_phy_my_0.083deg_static",
    "variables" : ["deptho"],
    "longitude" : [-60, -10], 
    "latitude" : [60, 40],
    "time" : [None, None],
    "depth": [0.49402499198913574, 5727.9169921875]
}

In [5]:
df = cm.read_dataframe(
    dataset_id=data_request["dataset_id"],
    variables=data_request["variables"],
    minimum_longitude=data_request["longitude"][0],
    maximum_longitude=data_request["longitude"][1],
    minimum_latitude=data_request["latitude"][0],
    maximum_latitude=data_request["latitude"][1],
    minimum_depth=data_request["depth"][0],
    maximum_depth=data_request["depth"][1],
    start_datetime=data_request["time"][0],
    end_datetime=data_request["time"][1]
)

INFO - 2024-08-06T17:19:38Z - Dataset version was not specified, the latest one was selected: "202311"
INFO - 2024-08-06T17:19:38Z - Dataset part was not specified, the first one was selected: "bathy"
INFO - 2024-08-06T17:19:38Z - Service was not specified, the default one was selected: "static-arco"


In [6]:
print(df.shape)
df.head()

(601, 1)


deptho
latitude longitude              
60.0     -60.000000  1786.468994
         -59.916668  1900.500977
         -59.833332  2029.793945
         -59.750000  2127.037109
         -59.666668  2191.720947

In [7]:
# remove the index columns corresponding to time, latitude and longitude
df = df.reset_index()
# drop na rows
df = df.dropna()

df.head()

,latitude,longitude,deptho
0,60.0,-60.000000,1786.468994
1,60.0,-59.916668,1900.500977
2,60.0,-59.833332,2029.793945
3,60.0,-59.750000,2127.037109
4,60.0,-59.666668,2191.720947


In [8]:
# function to coarse grain the data and make resolution same as Pisces data
def coarse_grain(df, features):
  """
    Parameters
    ----------
    df: pandas dataframe containing the data accessed from copernicus mariner
    features: name of the features in the dataframe

    Output
    ------
    a pandas dataframe with feature values for 0.25 deg x 0.25 deg resolution 

    """
  
  df["0_1"] = df["latitude"].to_numpy() - np.floor(df["latitude"])
  conditions = [df["0_1"] < 0.25,
                (df["0_1"] >= 0.25)  & (df["0_1"] < 0.5),
                (df["0_1"] >= 0.5)  & (df["0_1"] < 0.75),
                (df["0_1"] >= 0.75)  & (df["0_1"] < 1)]
  outputs = [0, 0.25, 0.5, 0.75]
  df['latitude'] = np.floor(df["latitude"]) + np.select(conditions, outputs)
  df = df.drop(columns=["0_1"])

  df["0_1"] = df["longitude"].to_numpy() - np.floor(df["longitude"])
  # if not redefined then conditions is based on latitude
  conditions = [df["0_1"] < 0.25,
              (df["0_1"] >= 0.25)  & (df["0_1"] < 0.5),
              (df["0_1"] >= 0.5)  & (df["0_1"] < 0.75),
              (df["0_1"] >= 0.75)  & (df["0_1"] < 1)]
  df['longitude'] = np.floor(df["longitude"]) + np.select(conditions, outputs)
  df = df.drop(columns=["0_1"])

  return df.groupby(["latitude","longitude"])[features].mean() 

In [9]:
# Need coarse grain the values because all these datasets have finer resolution than Pisces data
# Pisces data has resolution of 0.25 deg x 0.25 deg
df_cg = coarse_grain(df, df.columns[2:].tolist())
print(df_cg.shape)
df_cg.head()

(197, 1)


deptho
latitude longitude             
60.0     -60.00     1905.587891
         -59.75     2183.020996
         -59.50     2340.918701
         -59.25     2482.410400
         -59.00     2601.250000

In [12]:
df_cg.columns = ["bathy"]
df_cg.head()

bathy
latitude longitude             
60.0     -60.00     1905.587891
         -59.75     2183.020996
         -59.50     2340.918701
         -59.25     2482.410400
         -59.00     2601.250000

In [13]:
df_cg.to_csv(filename+".csv")